In [ ]:
# import libraries
from sklearn.neural_network import MLPRegressor
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras


from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras.layers import Flatten, Conv1D, MaxPooling1D, LSTM
from keras.activations import relu, elu, linear, softmax
from keras.callbacks import EarlyStopping, Callback
#from tensorflow.keras.optimizers import Adam, Nadam
from keras.losses import mean_squared_error, categorical_crossentropy, logcosh
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping
import time

#pip install keras-tuner --upgrade
import keras_tuner
from keras_tuner import RandomSearch




In [ ]:
# Check available devices
devices = tf.config.list_physical_devices('GPU')

if len(devices) > 0:
    print("GPU is available")
    for device in devices:
        print("Device name:", device.name)
else:
    print("No GPU is available")

In [ ]:
# processig data
def data_preprocess(dataset,target): 
    # Scaled data
    min_max_scaler = MinMaxScaler(feature_range = (0,1))
    np_scaled = min_max_scaler.fit_transform(dataset)
    X = pd.DataFrame(np_scaled)
    
    target_edit = pd.Series(target).values
    target_edit = target_edit.reshape(-1,1)
    np_scaled = min_max_scaler.fit_transform(target_edit)
    Y = pd.DataFrame(np_scaled)
    # Split data
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)
      
    scaler = StandardScaler()
    # Fit only to the training data
    scaler.fit(X_train)
        
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)
    return X_train, y_train, X_test, y_test

In [ ]:
data = pd.read_csv('/uufs/chpc.utah.edu/common/home/akaundal-group3/Vishal/NAM_dat.csv', header=0)
#data = pd.read_csv('/uufs/chpc.utah.edu/common/home/akaundal-group3/Vishal/NAM_dat.csv', header=0)
dataset=data.iloc[:,19:]
target=data.iloc[:, 4]
    
#preprocess data
X_train, y_train, X_test, y_test = data_preprocess(dataset, target)

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    #model.add(keras.layers.Flatten())
#     model.add(
#         keras.layers.Dense(
#             # Tune number of units.
#             units=hp.Int("units", min_value=32, max_value=512, step=32),
#             # Tune the activation function to use.
#             activation=hp.Choice("activation", ["relu","tanh"]),
#         )
#     )
#     # Tune whether to use dropout.
#     if hp.Boolean("dropout"):
#         model.add(keras.layers.Dropout(rate=0.25))
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(keras.layers.Dense(units=hp.Int('units_'+str(i),
                                           min_value=32,
                                           max_value=512),
                              activation='relu'))
    model.add(keras.layers.Dense(1, activation="linear"))
    # Define the optimizer learning rate as a hyperparameter.
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="mean_absolute_error",
        metrics=["mean_absolute_error"])
    return model


In [ ]:
tuner = keras_tuner.RandomSearch(
    build_model,
    objective=keras_tuner.Objective('val_mean_absolute_error','min'),
    max_trials=50,
    executions_per_trial=4,
    #distribution_strategy=tf.distribute.MirroredStrategy(),
    directory='project2',
    project_name='GS_KSdata2')
tuner.search_space_summary()

In [1]:
tuner = keras_tuner.BayesianOptimization(
    build_model,
    objective=keras_tuner.Objective('val_mean_absolute_error','min'),
    max_trials=50,
    executions_per_trial=4,
    #distribution_strategy=tf.distribute.MirroredStrategy(),
    directory='project2',
    project_name='GS_KSdata2')
tuner.search_space_summary()

NameError: name 'keras_tuner' is not defined

In [ ]:
tuner.search(X_train, y_train, epochs=15, validation_data=(X_test, y_test))

In [ ]:
tuner.results_summary()